# Explore here

In [1]:
# Your code here
import os
from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

def connect():
    global engine # Esto nos permite usar una variable global llamada "engine"
    # Un "connection string" es básicamente una cadena que contiene todas las credenciales de la base de datos juntas
    connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
    print("Starting the connection...")
    engine = create_engine(connection_string)
    engine.connect()
    return engine

engine = connect()

try:
    with engine.connect() as conn:
        # Ejecuta cada sentencia CREATE TABLE individualmente
        tablas = [
            """
            CREATE TABLE IF NOT EXISTS publishers(
                publisher_id INT NOT NULL,
                name VARCHAR(255) NOT NULL,
                PRIMARY KEY(publisher_id)
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS authors(
                author_id INT NOT NULL,
                first_name VARCHAR(100) NOT NULL,
                middle_name VARCHAR(50) NULL,
                last_name VARCHAR(100) NULL,
                PRIMARY KEY(author_id)
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS books(
                book_id INT NOT NULL,
                title VARCHAR(255) NOT NULL,
                total_pages INT NULL,
                rating DECIMAL(4, 2) NULL,
                isbn VARCHAR(13) NULL,
                published_date DATE,
                publisher_id INT NULL,
                PRIMARY KEY(book_id),
                CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id)
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS book_authors (
                book_id INT NOT NULL,
                author_id INT NOT NULL,
                PRIMARY KEY(book_id, author_id),
                CONSTRAINT fk_book FOREIGN KEY(book_id) REFERENCES books(book_id) ON DELETE CASCADE,
                CONSTRAINT fk_author FOREIGN KEY(author_id) REFERENCES authors(author_id) ON DELETE CASCADE
            );
            """
        ]
        for tabla in tablas:
            conn.execute(text(tabla))  # Encapsula cada sentencia con text()
        
        inserts = ["""
            INSERT INTO publishers(publisher_id, name) VALUES (1, 'O Reilly Media');
            INSERT INTO publishers(publisher_id, name) VALUES (2, 'A Book Apart');
            INSERT INTO publishers(publisher_id, name) VALUES (3, 'A K PETERS');
            INSERT INTO publishers(publisher_id, name) VALUES (4, 'Academic Press');
            INSERT INTO publishers(publisher_id, name) VALUES (5, 'Addison Wesley');
            INSERT INTO publishers(publisher_id, name) VALUES (6, 'Albert&Sweigart');
            INSERT INTO publishers(publisher_id, name) VALUES (7, 'Alfred A. Knopf');
          
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (1, 'Merritt', null, 'Eric');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (2, 'Linda', null, 'Mui');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (3, 'Alecos', null, 'Papadatos');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (4, 'Anthony', null, 'Molinaro');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (5, 'David', null, 'Cronin');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (6, 'Richard', null, 'Blum');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (7, 'Yuval', 'Noah', 'Harari');
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (8, 'Paul', null, 'Albitz');

         
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (1, 'Lean Software Development: An Agile Toolkit', 240, 4.17, '9780320000000', '2003-05-18', 5);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (2, 'Facing the Intelligence Explosion', 91, 3.87, null, '2013-02-01', 7);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (3, 'Scala in Action', 419, 3.74, '9781940000000', '2013-04-10', 1);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (4, 'Patterns of Software: Tales from the Software Community', 256, 3.84, '9780200000000', '1996-08-15', 1);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (5, 'Anatomy Of LISP', 446, 4.43, '9780070000000', '1978-01-01', 3);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (6, 'Computing machinery and intelligence', 24, 4.17, null, '2009-03-22', 4);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (7, 'XML: Visual QuickStart Guide', 269, 3.66, '9780320000000', '2009-01-01', 5);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (8, 'SQL Cookbook', 595, 3.95, '9780600000000', '2005-12-01', 7);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (9, 'The Apollo Guidance Computer: Architecture And Operation (Springer Praxis Books / Space Exploration)', 439, 4.29, '9781440000000', '2010-07-01', 6);
            INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (10, 'Minds and Computers: An Introduction to the Philosophy of Artificial Intelligence', 222, 3.54, '9780750000000', '2007-02-13', 7);

           
            INSERT INTO book_authors (book_id, author_id) VALUES (1, 1);
            INSERT INTO book_authors (book_id, author_id) VALUES (2, 8);
            INSERT INTO book_authors (book_id, author_id) VALUES (3, 7);
            INSERT INTO book_authors (book_id, author_id) VALUES (4, 6);
            INSERT INTO book_authors (book_id, author_id) VALUES (5, 5);
            INSERT INTO book_authors (book_id, author_id) VALUES (6, 4);
            INSERT INTO book_authors (book_id, author_id) VALUES (7, 3);
            INSERT INTO book_authors (book_id, author_id) VALUES (8, 2);
            INSERT INTO book_authors (book_id, author_id) VALUES (9, 4);
            INSERT INTO book_authors (book_id, author_id) VALUES (10, 1);
            """]
        conn.commit()  # Confirma los cambios después de crear todas las tablas
        print("¡Tablas creadas exitosamente!")

        for insert in inserts:
            conn.execute(text(insert))
        conn.commit() 
        print("Datos insertados exitosamente")

except Exception as e:
    print(f"Ocurrió un error: {e}")
    conn.rollback()

finally:
    engine.dispose()

Starting the connection...


¡Tablas creadas exitosamente!
Ocurrió un error: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "publishers_pkey"
DETAIL:  Key (publisher_id)=(1) already exists.

[SQL: 
            INSERT INTO publishers(publisher_id, name) VALUES (1, 'O Reilly Media');
            INSERT INTO publishers(publisher_id, name) VALUES (2, 'A Book Apart');
            INSERT INTO publishers(publisher_id, name) VALUES (3, 'A K PETERS');
            INSERT INTO publishers(publisher_id, name) VALUES (4, 'Academic Press');
            INSERT INTO publishers(publisher_id, name) VALUES (5, 'Addison Wesley');
            INSERT INTO publishers(publisher_id, name) VALUES (6, 'Albert&Sweigart');
            INSERT INTO publishers(publisher_id, name) VALUES (7, 'Alfred A. Knopf');
          
            INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (1, 'Merritt', null, 'Eric');
            INSERT INTO authors (author_id, first_name, middle_name, last_name

In [3]:
dataframe = pd.read_sql("Select * from books;", engine)
print(dataframe.describe())
print(dataframe)

        book_id  total_pages     rating  publisher_id
count  10.00000    10.000000  10.000000     10.000000
mean    5.50000   300.100000   3.966000      4.600000
std     3.02765   174.566415   0.289643      2.319004
min     1.00000    24.000000   3.540000      1.000000
25%     3.25000   226.500000   3.765000      3.250000
50%     5.50000   262.500000   3.910000      5.000000
75%     7.75000   434.000000   4.170000      6.750000
max    10.00000   595.000000   4.430000      7.000000
   book_id                                              title  total_pages  \
0        1        Lean Software Development: An Agile Toolkit          240   
1        2                  Facing the Intelligence Explosion           91   
2        3                                    Scala in Action          419   
3        4  Patterns of Software: Tales from the Software ...          256   
4        5                                    Anatomy Of LISP          446   
5        6               Computing machinery a